In [14]:
!pip install pydantic[dotenv]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [15]:
# Libraries
import pandas as pd
import requests
from config import settings
from data import API, SQLRepo

import sqlite3

# ETL

## Extract and Transform

### Extract
This part handles data extraction from Alpha Vintage API

In [16]:
currency_from = "USD"
currency_to = "KES"
output_size = "compact"

# make api url
url = (
    "https://www.alphavantage.co/query?"
    "function=FX_DAILY&"
    f"from_symbol={currency_from}&"
    f"to_symbol={currency_to}&"
    f"outputsize={output_size}&"
    f"apikey={settings.api_key}"
    )
print(type(url))

<class 'str'>


In [17]:
# request data from API
response = requests.get(url)

# check response status code
print(f"Response Code: {response.status_code}")
print(response.text[:400])

Response Code: 200
{
    "Meta Data": {
        "1. Information": "Forex Daily Prices (open, high, low, close)",
        "2. From Symbol": "USD",
        "3. To Symbol": "KES",
        "4. Output Size": "Compact",
        "5. Last Refreshed": "2023-06-23 21:55:00",
        "6. Time Zone": "UTC"
    },
    "Time Series FX (Daily)": {
        "2023-06-23": {
            "1. open": "140.33000",
            "2. high": "


In [18]:
# get the data
response_data = response.json()
print(response_data.keys())

# Extract 'Time Series FX (Daily)' from response_data
fx_data = response_data['Time Series FX (Daily)']
print(type(fx_data))

dict_keys(['Meta Data', 'Time Series FX (Daily)'])
<class 'dict'>


In [19]:
# see how the data is organised
fx_data["2023-06-23"]

{'1. open': '140.33000',
 '2. high': '140.35000',
 '3. low': '140.08000',
 '4. close': '140.33000'}

### Transform
Here, the data is transformed and presented as a clean DataFrame

In [20]:
# put data in a dataframe
df = pd.DataFrame.from_dict(fx_data, orient="index", dtype=float)
print(f"df_fx shape: {df.shape}")
print(df.info())
df.head()

df_fx shape: (100, 4)
<class 'pandas.core.frame.DataFrame'>
Index: 100 entries, 2023-06-23 to 2023-02-06
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   1. open   100 non-null    float64
 1   2. high   100 non-null    float64
 2   3. low    100 non-null    float64
 3   4. close  100 non-null    float64
dtypes: float64(4)
memory usage: 3.9+ KB
None


,1. open,2. high,3. low,4. close
2023-06-23,140.33,140.35,140.08,140.33
2023-06-22,140.08,140.35,140.08,140.33
2023-06-21,140.13,140.23,139.83,140.20
2023-06-20,140.03,140.18,139.78,140.13
2023-06-19,139.73,140.08,139.73,140.03


In [21]:
# convert df index to DateTime format
df.index = pd.to_datetime(df.index)

# Name index "date"
df.index.name = "date"

print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 100 entries, 2023-06-23 to 2023-02-06
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   1. open   100 non-null    float64
 1   2. high   100 non-null    float64
 2   3. low    100 non-null    float64
 3   4. close  100 non-null    float64
dtypes: float64(4)
memory usage: 3.9 KB
None


,1. open,2. high,3. low,4. close
date,,,,
2023-06-23,140.33,140.35,140.08,140.33
2023-06-22,140.08,140.35,140.08,140.33
2023-06-21,140.13,140.23,139.83,140.20
2023-06-20,140.03,140.18,139.78,140.13
2023-06-19,139.73,140.08,139.73,140.03


In [22]:
# clean column names
df.columns = [c.split(". ")[1] for c in df.columns]
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 100 entries, 2023-06-23 to 2023-02-06
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   open    100 non-null    float64
 1   high    100 non-null    float64
 2   low     100 non-null    float64
 3   close   100 non-null    float64
dtypes: float64(4)
memory usage: 3.9 KB
None


,open,high,low,close
date,,,,
2023-06-23,140.33,140.35,140.08,140.33
2023-06-22,140.08,140.35,140.08,140.33
2023-06-21,140.13,140.23,139.83,140.20
2023-06-20,140.03,140.18,139.78,140.13
2023-06-19,139.73,140.08,139.73,140.03


### Import Data Using a Function
This function symplifies the whole Extraction and Transformation step and allows user to parse in the currency-pair of their choice

In [23]:
def get_data(currency_from="USD", currency_to="KES", output_size="full"):
  # create url
  url = (
    "https://www.alphavantage.co/query?"
    "function=FX_DAILY&"
    f"from_symbol={currency_from}&"
    f"to_symbol={currency_to}&"
    f"outputsize={output_size}&"
    f"apikey={settings.api_key}"
    )
  # send request to API
  response = requests.get(url)

  # extract json data from response
  response_data = response.json()

  # put data in a dataframe
   ## data validation step
  if "Time Series FX (Daily)" not in response_data.keys():
    raise Exception(
        f"Invalid API call: check that currency_from sysmbol: '{currency_from}' and currency_to symbol: '{currency_to}' are correct"
    )

  fx_data = response_data['Time Series FX (Daily)']
  df = pd.DataFrame.from_dict(fx_data, orient="index", dtype=float)

  # convert index to datetime fomat: name index as 'date'
  df.index = pd.to_datetime(df.index)
  df.index.name = "date"

  # clean column names
  df.columns = [c.split(". ")[1] for c in df.columns]

  return df


In [24]:
#test get_data function
data = get_data("GBP", "USD", "compact")
print(data.info())
data.head()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 100 entries, 2023-06-23 to 2023-02-06
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   open    100 non-null    float64
 1   high    100 non-null    float64
 2   low     100 non-null    float64
 3   close   100 non-null    float64
dtypes: float64(4)
memory usage: 3.9 KB
None


,open,high,low,close
date,,,,
2023-06-23,1.27477,1.27489,1.26850,1.27125
2023-06-22,1.27677,1.28394,1.27252,1.27433
2023-06-21,1.27631,1.28023,1.26908,1.27661
2023-06-20,1.27949,1.28073,1.27132,1.27631
2023-06-19,1.28255,1.28370,1.27702,1.27941


We are going to use our function in a new python class as described in the data.py module

### Test API class

In [25]:
# instantiate 'API' instance

api = API()
print("api type:", type(api))

api type: <class 'data.API'>


In [26]:
c_from = "CHF"
c_to = "CAD"

# get data using your module
fx = api.get_api_data(currency_from=c_from, currency_to=c_to)

print(fx.info())
fx.head()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5000 entries, 2023-06-23 to 2004-04-16
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   open    5000 non-null   float64
 1   high    5000 non-null   float64
 2   low     5000 non-null   float64
 3   close   5000 non-null   float64
dtypes: float64(4)
memory usage: 195.3 KB
None


,open,high,low,close
date,,,,
2023-06-23,1.4678,1.4755,1.4634,1.4683
2023-06-22,1.4731,1.4756,1.4654,1.4688
2023-06-21,1.4730,1.4747,1.4681,1.4733
2023-06-20,1.4739,1.4752,1.4706,1.4729
2023-06-19,1.4747,1.4780,1.4700,1.4734


## Load
The following part is going to handle loading the resulting dataframe into a database and pulling the data out of the database for further processing/ analysis

### Load Data in SQL Database

In [27]:
# make a connection to sqlite3
conn = sqlite3.connect(settings.db_name, check_same_thread=False)
print(type(conn))

<class 'sqlite3.Connection'>


In [28]:
# read data to database
n_records = fx.to_sql(name=f"{c_from}{c_to}", con=conn, if_exists="replace")
print(f"Number of records updated is {n_records}")

Number of records updated is 5000


In [29]:
# function for loading data to the database
def load_data(connection, records, currency_from=c_from, currency_to=c_to, if_exists="fail"):
    table_name = f"{c_from}{c_to}"
    # read data to database
    n_records = records.to_sql(name=table_name, con=connection, if_exists=if_exists)

    return {
        "Table Name: " : table_name,
        "Number of records updated" : n_records
        }


## add this function to data.py as a method in a new class

In [30]:
# test load_data
load_data(connection=conn, records=fx, if_exists="replace")

{'Table Name: ': 'CHFCAD', 'Number of records updated': 5000}

### Pull data from database

In [31]:
# SQL query to be executed or a table name (sql)
query = f"SELECT * FROM {c_from}{c_to} limit 2500"
# database connection (con)
conn = conn
# read data from database
df = pd.read_sql(
    sql=query,
    con=conn,
    parse_dates=["date"],
    index_col="date"
    )

print(df.shape)
print(df.info())
df.head()

(2500, 4)
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2500 entries, 2023-06-23 to 2013-11-22
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   open    2500 non-null   float64
 1   high    2500 non-null   float64
 2   low     2500 non-null   float64
 3   close   2500 non-null   float64
dtypes: float64(4)
memory usage: 97.7 KB
None


,open,high,low,close
date,,,,
2023-06-23,1.4678,1.4755,1.4634,1.4683
2023-06-22,1.4731,1.4756,1.4654,1.4688
2023-06-21,1.4730,1.4747,1.4681,1.4733
2023-06-20,1.4739,1.4752,1.4706,1.4729
2023-06-19,1.4747,1.4780,1.4700,1.4734


In [32]:
# function to take data from sql database for further processsing

def get_sql_data(connection, limit=None):
    # define query
    if limit:
      query = f"SELECT * FROM {c_from}{c_to} limit {limit}"
    else:
      query = f"SELECT * FROM {c_from}{c_to}"

    # read data from database
    df = pd.read_sql(
        sql=query,
        con=connection,
        parse_dates=["date"],
        ).set_index(["date"])

    return df

## add this function to data.py module as a method

In [33]:
df_fx = get_sql_data(connection=conn, limit=90)
df_fx.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 90 entries, 2023-06-23 to 2023-02-20
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   open    90 non-null     float64
 1   high    90 non-null     float64
 2   low     90 non-null     float64
 3   close   90 non-null     float64
dtypes: float64(4)
memory usage: 3.5 KB


### Test the the SQLRepo() class

In [34]:
# Instantiate SQLRepo
repo = SQLRepo()
print(type(repo))

<class 'data.SQLRepo'>


In [35]:
# load data using repo
repo.load_data(records=fx,currency_from=c_from, currency_to=c_to, if_exists="replace")

{'Table Name: ': 'CHFCAD', 'Number of records updated': 5000}

In [36]:
# pull data from sql database using repo
data = repo.get_sql_data(currency_from=c_from, currency_to=c_to, limit=2500)
print(data.info())
data.head()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2500 entries, 2023-06-23 to 2013-11-22
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   open    2500 non-null   float64
 1   high    2500 non-null   float64
 2   low     2500 non-null   float64
 3   close   2500 non-null   float64
dtypes: float64(4)
memory usage: 97.7 KB
None


,open,high,low,close
date,,,,
2023-06-23,1.4678,1.4755,1.4634,1.4683
2023-06-22,1.4731,1.4756,1.4654,1.4688
2023-06-21,1.4730,1.4747,1.4681,1.4733
2023-06-20,1.4739,1.4752,1.4706,1.4729
2023-06-19,1.4747,1.4780,1.4700,1.4734
